# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [30]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [31]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_7.1.2025.pdf'
req = requests.get(url)

In [32]:
pdf = pdfplumber.open(BytesIO(req.content))

In [33]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [34]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [35]:
lines

[['Statewide Report by County - July 1, 2025',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'No Labels',
  'Inactive',
  'Total Active'],
 ['Aurora', '509', '2', '1,003', '232', '2', '101', '', '40', '1,849'],
 ['Beadle',
  '2,472',
  '27',
  '5,224',
  '1,375',
  '41',
  '1,204',
  '1',
  '742',
  '10,344'],
 ['Bennett', '667', '8', '728', '193', '2', '200', '', '172', '1,798'],
 ['Bon Homme', '874', '11', '2,426', '491', '', '240', '', '163', '4,042'],
 ['Brookings',
  '5,105',
  '108',
  '10,131',
  '3,669',
  '21',
  '2,870',
  '1',
  '836',
  '21,905'],
 ['Brown',
  '7,133',
  '104',
  '13,045',
  '3,594',
  '12',
  '2,011',
  '3',
  '379',
  '25,902'],
 ['Brule', '881', '12', '1,781', '465', '4', '221', '', '82', '3,364'],
 ['Buffalo', '531', '2', '154', '76', '4', '101', '', '116', '868'],
 ['Butte', '764', '55', '4,654', '872', '21', '665', '',

In [36]:
# report date is in the first line
report_date = lines.pop(0)

In [37]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2025-07-01


In [38]:
# save the PDF
outpath = Path('pdfs') / f'{report_date}.pdf'

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote pdfs/2025-07-01.pdf


In [39]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [40]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [41]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'no_labels',
 'inactive',
 'total active']

In [42]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [43]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [44]:
data[0]

{'county': 'Aurora',
 'democratic': 509,
 'libertarian': 2,
 'republican': 1003,
 'independent': 232,
 'other': 2,
 'npa': 101,
 'no_labels': '',
 'inactive': 40,
 'date': '2025-07-01'}

In [45]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [46]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/cjwinchester/south-dakota-voter-registration-data/data/2025-07-01.csv


In [47]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
